# This is the source code used in Ryerson-Intel project
Author: Alice

Created: Dec 2019

Modified for EMD analysis: Mar 2, 2020

Classification for EMD Triad

In [1]:
# Mount the google drive to get the feature files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Import system packages
import os
import glob

# Importing packages
import pandas as pd
import numpy as np
from numpy import set_printoptions

# Preparing dataset
from sklearn.preprocessing import MinMaxScaler
import random

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


# Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Performance Measure
from sklearn import metrics
import statistics

# Plotting
import matplotlib.pyplot as plt

In [0]:
# Control setting to see which types of data
FCTOL = 'FC15Per' #FC10Per or FC15Per
SEG   = 'Triad'   #FixedFrameSize, Triad, Delta, or '' 

# Set the folder according to segmentation: '16K Fixed Frame Size', '16K Triad', '4k', and 'EMDDeltaEMD'
if SEG=='FixedFrameSize':
  segmentation = '16K Fixed Frame Size'
elif SEG == 'Triad':
  segmentation = '16K Triad' 
elif SEG == 'Delta':
  segmentation = 'EMDDeltaEMD'
else:
  segmentation = ''

## Preparing the Dataset
Loading the feature csv file into arrays and normalized the features to range of [0,1]. There are 49 signals in total with 21 stress signals and 28 non-stress signals

In [4]:
# Setting folder directory
datapath = '/content/drive/My Drive/2019 DDK Paper/Journal/Features/EMD/GITA Features/'
 
# The actual datafile is in Statistical Summary folder
datapath = datapath + segmentation + '/Statistical Summary/'


filename = SEG+'*'+FCTOL+'*.csv'
#addrs = glob.glob(datapath+'*.csv')
addrs = glob.glob(datapath+'*'+filename)

print(segmentation)
print(os.path.basename(addrs[0]))
print(os.path.basename(addrs[1]))



16K Triad
EMDAmpHCTriadFC15Per.csv
EMDAmpPDTriadFC15Per.csv


#### Complete set of features

In [5]:
# Construct dataframe for analysis

# Importing data matrices
HCaddr = [addr for addr in addrs if 'HC' in addr ]
PDaddr = [addr for addr in addrs if 'PD' in addr ]
dfHCOriginal = pd.read_csv(HCaddr[0])
dfPDOriginal = pd.read_csv(PDaddr[0])

print('HC matrix dim: ', dfHCOriginal.shape)
print('PD matrix dim: ', dfPDOriginal.shape)


# Combining the HC and PD dataframes
dfHCPD = pd.concat([dfHCOriginal,dfPDOriginal], axis=0)
dfHCPD = dfHCPD.drop(columns=['PowerOBW(W)Mean', 'Power3dB(W)Mean',
                                'PowerOBW(W)Std', 'Power3dB(W)Std',
                                'PowerOBW(W)Skew', 'Power3dB(W)Skew',
                                'PowerOBW(W)Kurt', 'Power3dB(W)Kurt'])

Labels = np.reshape([np.ones([50,1]), np.zeros([50,1])], (-1,1)) # Label for HC=1 and PD=0

# features
colnames = dfHCPD.columns
hcpdfeatures = np.array(dfHCPD.iloc[:,1:].values)

print('Features: ', colnames)
print('HC matrix dim: ', dfHCPD.shape)
print('size of HCPD features: ', hcpdfeatures.shape)



HC matrix dim:  (50, 282)
PD matrix dim:  (50, 282)
Features:  Index(['subject', 'segmentNum', 'numIMFMean', 'OBW1Mean', 'OBW2Mean',
       'OBW3Mean', 'OBW4Mean', 'OBW5Mean', 'OBW6Mean', 'OBW7Mean',
       ...
       'FCenter8Kurt', 'FCRatio1Kurt', 'FCRatio2Kurt', 'FCRatio3Kurt',
       'FCRatio4Kurt', 'FCRatio5Kurt', 'FCRatio6Kurt', 'SNRdBKurt',
       'AmplificationKurt', 'Length(s)Kurt'],
      dtype='object', length=274)
HC matrix dim:  (100, 274)
size of HCPD features:  (100, 273)


#### Subset of different measure features

In [32]:
# separating headers for different types of measures
obwheaders = [colname for colname in colnames if 'OBW' in colname]+[colname for colname in colnames if 'obw' in colname]
p3dbheaders = [colname for colname in colnames if '3dB' in colname]
fcheaders = [colname for colname in colnames if 'FC' in colname]
measheaders = obwheaders+p3dbheaders+fcheaders
otherheaders = [colname for colname in colnames if colname not in measheaders]

obwheaders = ['subject']+obwheaders
p3dbheaders = ['subject']+p3dbheaders
fcheaders = ['subject']+fcheaders

print(len(obwheaders),obwheaders)
print(len(p3dbheaders),p3dbheaders)
print(len(fcheaders),fcheaders)
print(len(otherheaders),otherheaders)


# Creating dataframes and matrices for all subsets
dfobw = dfHCPD[obwheaders]
df3db = dfHCPD[p3dbheaders]
dffc = dfHCPD[fcheaders]
dfother = dfHCPD[otherheaders]

obwfeat = np.array(dfobw.iloc[:,1:].values)
p3dbfeat = np.array(df3db.iloc[:,1:].values)
fcfeat = np.array(dffc.iloc[:,1:].values)
otherfeat = np.array(dfother.iloc[:,1:].values)

print(obwfeat.shape, p3dbfeat.shape, fcfeat.shape, otherfeat.shape)



145 ['subject', 'OBW1Mean', 'OBW2Mean', 'OBW3Mean', 'OBW4Mean', 'OBW5Mean', 'OBW6Mean', 'OBW7Mean', 'OBW8Mean', 'OBW9Mean', 'OBW10Mean', 'OBWRatio1Mean', 'OBWRatio2Mean', 'OBWRatio3Mean', 'OBWRatio4Mean', 'OBWRatio5Mean', 'OBWRatio6Mean', 'OBWRatio7Mean', 'OBWRatio8Mean', 'OBWRatio9Mean', 'OBWRatio10Mean', 'OBW1Std', 'OBW2Std', 'OBW3Std', 'OBW4Std', 'OBW5Std', 'OBW6Std', 'OBW7Std', 'OBW8Std', 'OBW9Std', 'OBW10Std', 'OBWRatio1Std', 'OBWRatio2Std', 'OBWRatio3Std', 'OBWRatio4Std', 'OBWRatio5Std', 'OBWRatio6Std', 'OBWRatio7Std', 'OBWRatio8Std', 'OBWRatio9Std', 'OBWRatio10Std', 'OBW1Skew', 'OBW2Skew', 'OBW3Skew', 'OBW4Skew', 'OBW5Skew', 'OBW6Skew', 'OBW7Skew', 'OBW8Skew', 'OBW9Skew', 'OBW10Skew', 'OBWRatio1Skew', 'OBWRatio2Skew', 'OBWRatio3Skew', 'OBWRatio4Skew', 'OBWRatio5Skew', 'OBWRatio6Skew', 'OBWRatio7Skew', 'OBWRatio8Skew', 'OBWRatio9Skew', 'OBWRatio10Skew', 'OBW1Kurt', 'OBW2Kurt', 'OBW3Kurt', 'OBW4Kurt', 'OBW5Kurt', 'OBW6Kurt', 'OBW7Kurt', 'OBW8Kurt', 'OBW9Kurt', 'OBW10Kurt', 'OBWRat

In [36]:
# Normalizing the features into [0,1]
## Complete feature
scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
rescaledfeat = np.array(scaler.fit_transform(hcpdfeatures)) # fit the trainning feature X into scaler

set_printoptions(precision=3) # how many decimal places.
print(rescaledfeat[0:5,:])

# normalize other features
rescaledobw = np.array(scaler.fit_transform(obwfeat))
rescaled3db = np.array(scaler.fit_transform(p3dbfeat))
rescaledfc = np.array(scaler.fit_transform(fcfeat))
rescaledother = np.array(scaler.fit_transform(otherfeat))
print(rescaledobw[0,0:10])
print(rescaled3db[0,0:10])
print(rescaledfc[0,0:10])
print(rescaledother[0,0:10])



[[0.278 0.49  0.495 ... 0.426 0.399 0.115]
 [0.667 0.449 0.588 ... 0.014 0.352 0.429]
 [0.333 0.626 0.011 ... 0.508 0.692 0.082]
 [0.167 0.592 0.636 ... 0.265 0.065 0.035]
 [0.5   0.413 0.878 ... 0.751 0.637 0.418]]
[0.495 0.489 0.4   0.539 0.478 0.377 0.469 0.489 0.347 0.335]
[0.454 0.614 0.743 0.606 0.432 0.403 0.216 0.11  0.058 0.191]
[0.495 0.389 0.622 0.629 0.526 0.459 0.428 0.378 0.178 0.235]
[0.278 0.49  0.125 0.02  0.185 0.49  0.667 0.025 0.106 0.453]


## Classification 
The classifiers tested in this project include Decision Tree, Each classifier uses 5-fold for validation.

In [0]:
# Classifiers
# Decision Tree
DT = DecisionTreeClassifier(random_state=0)
# Gradient Boost
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

# Support Vector Machine
SVM = SVC(random_state = 0, gamma='scale',probability = True)

# Linear Discrimant Analysis
LDA = LinearDiscriminantAnalysis()

# Gaussian Naive Bayes
GNB = GaussianNB()



In [0]:
# Classificaiton function with K-Fold cross-validation with user define iterations to get the standard deviation 

def classify(model, Iterations, num_folds, rescaledX, y):
    acc = []
    pre = []
    rec = []
    rocauc = []
    f1 = []
    Skip = False

    bestAcc = 0
    beststd = 0



    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        results = cross_val_score(model, rescaledX, y.ravel(), cv = kfold)
        if results.mean()>bestAcc:
          bestAcc = results.mean()
          beststd = results.std()
          print(results)

        for train_index, test_index in kfold.split(rescaledX):
            X_train, X_test = rescaledX[train_index], rescaledX[test_index]
            y_train, y_test = y[train_index], y[test_index]

            y_total = sum(y_test)
            if (y_total == 0) or (y_total == len(y_test)):
                Skip = True
                
            if not Skip:
                # perform training and testing
                if model == SVC:
                    model.fit(X_train,y_train)
                else:
                    model.fit(X_train,y_train.ravel())
                #dtscores = DT.score(X_test,y_test)
                yPred = model.predict(X_test)


                # record performance
                acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
                pre = np.append(pre,metrics.precision_score(y_test,
                                                            yPred, 
                                                            pos_label=1, 
                                                            average='macro', 
                                                            labels=np.unique(yPred)))
                rec = np.append(rec,metrics.recall_score(y_test,
                                                         yPred, 
                                                         pos_label=1, 
                                                         average='macro', 
                                                         labels=np.unique(yPred)))
                rocauc = np.append(rocauc, metrics.roc_auc_score(y_test, 
                                                                 yPred,
                                                                 average='macro'))
                f1 = np.append(f1, metrics.f1_score(y_test,yPred))
    return acc, pre, rec, rocauc, f1, bestAcc, beststd
    

    

### Parameter setting
The classification are set to 100 iteration using 5-fold cross validation.

NOTE: You might encounter single predicted labels instead of 2 class labels. This will return an error. The easiest work around is to run the cell again to shuffle tthe data. 

In [0]:
Iterations = 200
num_folds = 10


### Support Vector Machine

In [0]:
print('Support Vector Machine')
acc, pre, rec, rocauc, f1, bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaledfeat, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)

Support Vector Machine
[0.6 0.6 0.7 0.7 0.7 0.9 0.5 0.7 0.8 0.8]
[0.9 0.7 0.9 0.5 0.6 0.9 0.5 0.6 0.7 0.9]
[0.7 0.5 0.6 0.9 0.9 0.8 0.9 0.5 0.7 0.7]
[0.5 0.6 0.7 0.9 0.6 0.9 0.7 0.7 0.9 0.8]
[0.5 0.9 0.8 0.8 0.8 0.8 0.5 0.8 0.7 0.7]
Accuracy (mean, std): 0.688936170212766 0.1365307910248872
Precision (mean, std): 0.7054732353934482 0.14550688569540102
Recall (mean, std): 0.697041117865586 0.14550688569540102
Area under the Receiver Operating Characteristic Curve 0.697041117865586 0.13893286192561755
F1-score: 0.7060179286712454 0.1424253887887262
best iteration accuracy:  0.7300000000000001 0.12688577540449522


In [0]:
# Classificaiton function with K-Fold cross-validation with user define iterations to get the standard deviation 

def findbestresults(model, Iterations, num_folds, rescaledX, y):
    
    Skip = False

    bestAcc = 0

    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        results = cross_val_score(model, rescaledX, y.ravel(), cv = kfold)
        if results.mean()>bestAcc:
          bestAcc = results.mean()
          
          print(results)

          acc = []
          pre = []
          rec = []
          rocauc = []
          f1 = []

          for train_index, test_index in kfold.split(rescaledX):
              X_train, X_test = rescaledX[train_index], rescaledX[test_index]
              y_train, y_test = y[train_index], y[test_index]

              y_total = sum(y_test)
              if (y_total == 0) or (y_total == len(y_test)):
                  Skip = True
                  
              if not Skip:
                  # perform training and testing
                  if model == SVC:
                      model.fit(X_train,y_train)
                  else:
                      model.fit(X_train,y_train.ravel())
                  #dtscores = DT.score(X_test,y_test)
                  yPred = model.predict(X_test)


                  # record performance
                  acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
                  pre = np.append(pre,metrics.precision_score(y_test,
                                                              yPred, 
                                                              pos_label=1, 
                                                              average='macro', 
                                                              labels=np.unique(yPred)))
                  rec = np.append(rec,metrics.recall_score(y_test,
                                                          yPred, 
                                                          pos_label=1, 
                                                          average='macro', 
                                                          labels=np.unique(yPred)))
                  rocauc = np.append(rocauc, metrics.roc_auc_score(y_test, 
                                                                  yPred,
                                                                  average='macro'))
                  f1 = np.append(f1, metrics.f1_score(y_test,yPred))
    return acc, pre, rec, rocauc, f1


Iterations = 200
num_folds = 10
seed =14

print('Support Vector Machine')
acc, pre, rec, rocauc, f1 = findbestresults(SVM, Iterations, num_folds,  rescaledfeat, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))


Support Vector Machine
[0.6 0.7 0.7 0.7 0.7 0.4 0.9 0.7 0.7 0.8]
[0.6 0.7 0.8 0.5 0.5 0.8 0.7 0.8 0.8 0.8]
[0.6 0.8 0.6 0.8 0.7 0.7 0.7 0.7 0.8 0.7]
[0.8 0.4 0.8 0.6 0.9 0.8 0.7 0.8 0.6 0.7]
[0.7 0.6 0.9 0.9 0.7 0.8 0.8 0.5 0.4 0.9]
[0.5 0.9 0.7 0.6 0.8 0.7 0.8 0.8 0.7 0.9]
Accuracy (mean, std): 0.74 0.1264911064067352
Precision (mean, std): 0.7622222222222222 0.11085966920004785
Recall (mean, std): 0.74 0.11085966920004785
Area under the Receiver Operating Characteristic Curve 0.74 0.1296061149172195
F1-score: 0.7589252414252414 0.11742815916565502


#### Performance of different measure type

In [44]:
Iterations = 200
num_folds = 10

# OBW features
print('Support Vector Machine on OBW features')
acc, pre, rec, rocauc, f1, bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaledobw, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)



Support Vector Machine on OBW features
[0.8 0.6 0.9 0.8 0.7 0.6 0.7 0.5 0.7 0.8]
[0.8 0.9 0.3 0.6 0.7 0.8 1.  0.7 0.5 0.8]
[0.6 0.9 0.6 0.9 0.8 0.6 0.6 0.7 0.9 0.7]
[0.8 0.5 0.8 0.8 0.5 0.8 0.7 0.7 0.8 0.9]
[0.8 0.9 0.4 1.  0.8 0.9 0.7 0.8 0.5 0.7]
Accuracy (mean, std): 0.6833333333333333 0.14261568300277527
Precision (mean, std): 0.6975668337510443 0.15439154608367964
Recall (mean, std): 0.6911497493734335 0.15439154608367964
Area under the Receiver Operating Characteristic Curve 0.6911497493734337 0.1521060799711215
F1-score: 0.7002793989636095 0.15181249727214538
best iteration accuracy:  0.7500000000000001 0.1746424919657298


In [45]:
# P3dB features
print('Support Vector Machine on 3dB features')
acc, pre, rec, rocauc, f1, bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaled3db, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)

Support Vector Machine on 3dB features
[0.6 0.7 0.4 0.8 0.7 0.8 0.7 0.7 0.7 0.8]
[0.6 0.7 0.7 0.8 0.8 0.8 0.6 0.8 0.7 0.6]
[0.7 0.6 0.5 0.9 0.5 0.8 0.7 0.9 0.7 0.9]
[0.7 0.6 0.8 0.7 0.8 0.8 0.6 0.8 0.7 0.8]
Accuracy (mean, std): 0.6664536741214058 0.13702147008298063
Precision (mean, std): 0.6930485825853238 0.14555466661492641
Recall (mean, std): 0.684428089659719 0.14555466661492641
Area under the Receiver Operating Characteristic Curve 0.6796357573913485 0.1439022050905423
F1-score: 0.6919886415657887 0.14808259035567803
best iteration accuracy:  0.73 0.07810249675906658


In [46]:
# FC features
print('Support Vector Machine on fc features')
acc, pre, rec, rocauc, f1, bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaledfc, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)

Support Vector Machine on fc features
[0.4 0.4 1.  0.8 0.6 0.5 0.3 0.8 0.8 0.7]
[0.8 0.6 0.7 0.9 0.6 0.3 0.6 0.6 0.7 0.7]
[0.8 0.6 0.6 0.9 0.5 0.7 0.5 0.6 0.7 0.7]
[0.9 0.5 0.7 0.6 0.7 0.5 0.6 0.5 0.7 1. ]
Accuracy (mean, std): 0.6184466019417476 0.1415920438825427
Precision (mean, std): 0.6381116119586994 0.15821582708208118
Recall (mean, std): 0.6362883341038681 0.15821582708208118
Area under the Receiver Operating Characteristic Curve 0.6338611496378487 0.14462871232802907
F1-score: 0.6411201123579764 0.15422705775551113
best iteration accuracy:  0.67 0.16155494421403513


In [47]:
# Other features
print('Support Vector Machine')
acc, pre, rec, rocauc, f1, bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaledother, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)

Support Vector Machine
[0.5 0.8 0.6 0.7 0.7 0.9 0.4 0.9 0.6 0.6]
[0.6 0.8 0.7 0.7 0.8 0.7 0.6 1.  0.4 0.6]
Accuracy (mean, std): 0.6278236914600551 0.14985242018384043
Precision (mean, std): 0.6374010669465214 0.15893423940372273
Recall (mean, std): 0.6382892780620053 0.15893423940372273
Area under the Receiver Operating Characteristic Curve 0.6341570466570466 0.16143973825308994
F1-score: 0.6173643390585539 0.18604982268687945
best iteration accuracy:  0.6900000000000001 0.15132745950421556
